# Tree Visualization

## Package Imports

In [1]:
import cv2
import os
import math

import bayes_tree # contains bayes tree class
from bayes_tree import tree
import config as cfg # contains unit lookups/helper functions/credentials

In [2]:
def clean_prediction(pred):
    clusters = [0,1,2,3,4,5,6,7,8,9,10,11,12]
    pred_freq = {n: 0 for n in clusters}
    for vote in pred:
        total = sum(vote[0].values())
        for key in vote[0].keys():
            pred_freq[key] += vote[0][key] / total * vote[1]         
    return pred_freq

## Object Setup

In [3]:
all_obs,all_unique_ids = cfg.from_pickle('pickles/cleaned_obs.pickle')
tree = cfg.from_pickle('pickles/memo_tree.pickle')
blind_prob = cfg.from_pickle('pickles/blind_prob.pickle')

In [4]:
live_id = ('tvt', 314) 
live_index = all_unique_ids.index(live_id)
live_obs = all_obs[live_index]

## Render Radial Trees

In [5]:
tree.reset()
raw_prediction = []
img_files = []

node_command = "node --max-old-space-size=8192 viz/vg2png.js" #vg2png is a vega script to make vega images locally
vega_grammar = "viz/sctree-radial.vg.json"
image_folder = "viz/img/"
image_ext = ".png"

json_data_path = "viz/data/live.json" # vega grammar file references live.json for data to render

# Render Memorized Tree

tree.export_json(json_data_path) # vega grammar file references live.json for data to render

file_name = image_folder + "start" + image_ext
print(f'Rendering {file_name}')
os.system(node_command + " " + vega_grammar + " " + file_name)
img_files.append(file_name)
print('-------------------------')

# Render tree for each observation
for obs in live_obs:        
    unit_quant = obs[0][1]
    unit_name = cfg.tree_choices[obs[0][0]]
    unit_min = math.floor(obs[1]/60)
    unit_sec = str(obs[1] % 60).zfill(2)
    unit_string = f'{unit_quant}x_{unit_name}_{unit_min}m_{unit_sec}s'
    
    file_name = image_folder + unit_string + image_ext
    print(f'Rendering {file_name}')
    
    tree.update_tree(obs)
    tree.export_json(json_data_path)
    os.system(node_command + " " + vega_grammar + " " + file_name)
    img_files.append(file_name)
    raw_prediction.append(tree.predict(cfg.TIME-30*24,'cluster'))
    print('-------------------------')

Rendering viz/img/start.png
-------------------------
Rendering viz/img/1x_Terran_SCV_2m_40s.png
Freq Check: 0.9999999999999725
-------------------------
Rendering viz/img/1x_Terran_Marine_3m_23s.png
Freq Check: 1.000000000000054
-------------------------
Rendering viz/img/2x_Terran_Marine_3m_24s.png
Freq Check: 0.9999999999999544
-------------------------
Rendering viz/img/3x_Terran_Marine_3m_28s.png
Freq Check: 1.00000000000005
-------------------------
Rendering viz/img/1x_Terran_Barracks_3m_29s.png
Freq Check: 0.9999999999999469
-------------------------
Rendering viz/img/1x_Terran_Factory_3m_31s.png
Freq Check: 0.9999999999999603
-------------------------
Rendering viz/img/1x_Terran_SupplyDepot_3m_31s.png
Freq Check: 1.0000000000001104
-------------------------
Rendering viz/img/2x_Terran_SupplyDepot_3m_31s.png
Freq Check: 0.9999999999999124
-------------------------
Rendering viz/img/1x_Terran_CommandCenter_3m_32s.png
Freq Check: 1.00000000000007
-------------------------
Renderi

In [6]:
clean_predictions = []
for index,pred in enumerate(raw_prediction):
    clean_predictions.append(clean_prediction(pred))

In [11]:
#quick movie of the renders playing sequentially

video_name = 'video.avi'

frame = cv2.imread(img_files[0])
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 2, (width,height))

for image in img_files:
    video.write(cv2.imread(image))

cv2.destroyAllWindows()
video.release()